In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import librosa
import librosa.display
import sounddevice as sd
sd.default.samplerate = 16000
import torch
from mlp64 import data
from mlp64 import st
import torchaudio as ta
import matplotlib.pyplot as plt
from IPython.display import Audio

import scipy.fft as spfft
librosa.set_fftlib(spfft)

pd.set_option('display.max_columns', 30)

In [36]:
path = Path("/home/tim/Desktop/MLP64/dataset/nsynth-test/")

scontent = 'keyboard_acoustic_004-079-127.wav'
sstyle = 'brass_acoustic_046-059-075.wav'
others = ['bass_electronic_018-022-100.wav', 'flute_acoustic_002-107-100.wav']

In [37]:
npdata = data.load_files(path / "audio", [scontent, sstyle] + others)

In [55]:
# 16000 * 4 = 64000 samples
# 16000 samples per second
# e.g. 50ms are in 16000/20 = 800 samples
signal_length = 64000
sr = 16000
frame_size = 93/1000
frame_stride = 0.01
frame_length, frame_step = frame_size * sr, frame_stride * sr
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
print("Frame length:", frame_length, "Frame stepsize:", frame_step, "Num. frames:", num_frames)

Frame length: 1488 Frame stepsize: 160 Num. frames: 391


In [20]:
def mstofrac(x):
    x/1000

0.05

In [73]:
def stft_params(sr, frame_ms, hop_ms, signal_length):
    # Librosa stft uses signal_length/hop_length to determine number of frames, doesn't subtract frame_length first
    # Always truncate towards 0
    n_fft = int(sr * frame_ms/1000)
    hop_length = int(sr * hop_ms/1000)
    num_frames = int(np.ceil(signal_length/hop_length))
    num_bins = int(1 + n_fft/2)
    return n_fft, hop_length, num_bins, num_frames

In [80]:
def mstobin(ms, n_fft, hop_length, sr=16000):
    ms_samples = ms/1000 * sr
    return (ms_samples)/hop_length

In [83]:
mstobin(200, 800, 160)

20.0

In [32]:
mstobin(2000, 800, 160, 16000)

195.0

In [44]:
?librosa.stft

Signature:
librosa.stft(
    y,
    n_fft=2048,
    hop_length=None,
    win_length=None,
    window='hann',
    center=True,
    dtype=<class 'numpy.complex64'>,
    pad_mode='reflect',
)
Docstring:
Short-time Fourier transform (STFT). [1]_ (chapter 2)

The STFT represents a signal in the time-frequency domain by
computing discrete Fourier transforms (DFT) over short overlapping
windows.

This function returns a complex-valued matrix D such that

- `np.abs(D[f, t])` is the magnitude of frequency bin `f`
  at frame `t`, and

- `np.angle(D[f, t])` is the phase of frequency bin `f`
  at frame `t`.

The integers `t` and `f` can be converted to physical units by means
of the utility functions `frames_to_sample` and `fft_frequencies`.

.. [1] M. Müller. "Fundamentals of Music Processing." Springer, 2015


Parameters
----------
y : np.ndarray [shape=(n,)], real-valued
    input signal

n_fft : int > 0 [scalar]
    length of the windowed signal after padding with zeros.
    The number of rows

In [64]:
64000/167

383.23353293413174

In [71]:
hop_length = 167
n_fft = 261
print(1 + n_fft/2)
c = librosa.stft(npdata[:, 1], n_fft=n_fft, hop_length=hop_length)
print(c.shape, np.prod(c.shape))
ic = librosa.istft(c, hop_length=hop_length)
print(ic.shape)
sd.play(ic)

131.5
(131, 384) 50304
(63961,)


# even n_fft -> odd number of frames and vice versa
n_frames = ceil(signal_length)/hop_length

In [53]:
64000/160

400.0